In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\venka\\Desktop'

In [3]:
import nltk
import re
import numpy as np # linear algebra
import pandas as pd # data processing
import random
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize


#import pandas_profiling

%matplotlib inline

In [4]:
data=pd.read_csv("reviews.csv")

In [5]:
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [6]:
df=data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Product Name  413840 non-null  object 
 1   Brand Name    348669 non-null  object 
 2   Price         407907 non-null  float64
 3   Rating        413840 non-null  int64  
 4   Reviews       413778 non-null  object 
 5   Review Votes  401544 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 18.9+ MB


In [7]:
df.describe()

,Price,Rating,Review Votes
count,407907.000000,413840.000000,401544.000000
mean,226.867155,3.819578,1.507237
std,273.006259,1.548216,9.163853
min,1.730000,1.000000,0.000000
25%,79.990000,3.000000,0.000000
50%,144.710000,5.000000,0.000000
75%,269.990000,5.000000,1.000000
max,2598.000000,5.000000,645.000000


In [8]:
df=df[['Reviews','Rating']]
df.head()

,Reviews,Rating
0,I feel so LUCKY to have found this used (phone...,5
1,"nice phone, nice up grade from my pantach revu...",4
2,Very pleased,5
3,It works good but it goes slow sometimes but i...,4
4,Great phone to replace my lost phone. The only...,4


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413840 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.3+ MB


In [10]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413778 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  413778 non-null  object
 1   Rating   413778 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [11]:
df=df[df['Rating']!=3]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382015 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  382015 non-null  object
 1   Rating   382015 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.7+ MB


In [12]:
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382015 entries, 0 to 382014
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Reviews  382015 non-null  object
 1   Rating   382015 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ MB


In [13]:
df['sentiment']=np.where(df['Rating'] > 3, 1, 0)
df.head()

,Reviews,Rating,sentiment
0,I feel so LUCKY to have found this used (phone...,5,1
1,"nice phone, nice up grade from my pantach revu...",4,1
2,Very pleased,5,1
3,It works good but it goes slow sometimes but i...,4,1
4,Great phone to replace my lost phone. The only...,4,1


In [14]:
df.tail()

,Reviews,Rating,sentiment
382010,good rugged phone that has a long-lasting batt...,4,1
382011,used hard,1,0
382012,another great deal great price,5,1
382013,Passes every drop test onto porcelain tile!,5,1
382014,Only downside is that apparently Verizon no lo...,4,1


In [24]:
df

,Reviews,Rating,sentiment
0,I feel so LUCKY to have found this used (phone...,5,1
1,"nice phone, nice up grade from my pantach revu...",4,1
2,Very pleased,5,1
3,It works good but it goes slow sometimes but i...,4,1
4,Great phone to replace my lost phone. The only...,4,1
...,...,...,...
382010,good rugged phone that has a long-lasting batt...,4,1
382011,used hard,1,0
382012,another great deal great price,5,1
382013,Passes every drop test onto porcelain tile!,5,1


In [25]:
df

,Reviews,Rating,sentiment
0,I feel so LUCKY to have found this used (phone...,5,1
1,"nice phone, nice up grade from my pantach revu...",4,1
2,Very pleased,5,1
3,It works good but it goes slow sometimes but i...,4,1
4,Great phone to replace my lost phone. The only...,4,1
...,...,...,...
382010,good rugged phone that has a long-lasting batt...,4,1
382011,used hard,1,0
382012,another great deal great price,5,1
382013,Passes every drop test onto porcelain tile!,5,1


In [1]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [3]:
data=[]
for i in range(0,1000):
    review=df["Reviews"][i]
    review=re.sub('[^a-zA-Z]',' ',review)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    data.append(review)

NameError: name 'df' is not defined

In [30]:
data

['feel lucki found use phone us use hard phone line someon upgrad sold one son like old one final fell apart year want upgrad thank seller realli appreci honesti said use phone recommend seller highli would',
 'nice phone nice grade pantach revu clean set easi set never android phone fantast say least perfect size surf social media great phone samsung',
 'pleas',
 'work good goe slow sometim good phone love',
 'great phone replac lost phone thing volum button work still go set adjust job elig upgrad phone thaank',
 'alreadi phone problem know state use dang state charg wish would read comment would purchas item crack side damag good tri charg anoth way work request money back get money back sign unhappi custom',
 'charg port loos got solder need new batteri well later includ cost purchas usabl phone phone sold state',
 'phone look good stay charg buy new batteri still stay charg long trash money lost never buy person',
 'origin use samsung galaxi sprint want return back samsung epic g 

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
cv=CountVectorizer(max_features=20000,min_df=5,ngram_range=(1,2))

In [44]:
X=cv.fit_transform(df['Reviews'])

In [45]:
X.shape

(382015, 20000)

In [46]:
y=df['sentiment'].values

In [47]:
y.shape

(382015,)

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [49]:
X_train

<305612x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 13848947 stored elements in Compressed Sparse Row format>

In [ ]:
from keras.models import Sequential